In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [4]:
df.shape

(891, 12)

In [5]:
df = df.drop(columns=["PassengerId","Name","Ticket","Cabin"])

In [6]:
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [7]:
X = df.drop(columns="Survived")

In [8]:
y=df["Survived"]

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,
                                                 random_state=42)

In [11]:
X_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
445,1,male,4.0,0,2,81.8583,S
650,3,male,NaN,0,0,7.8958,S
172,3,female,1.0,1,1,11.1333,S
450,2,male,36.0,1,2,27.7500,S
314,2,male,43.0,1,1,26.2500,S
...,...,...,...,...,...,...,...
106,3,female,21.0,0,0,7.6500,S
270,1,male,NaN,0,0,31.0000,S
860,3,male,41.0,2,0,14.1083,S
435,1,female,14.0,1,2,120.0000,S


# Library

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# Step to create a pipeline
## 1.Dealing Missing values in dataset.
## 2.Deal with Categorical variable.(OnehotEncoding,OrdinalEncoder,LabelEncoder).
## 3. Feature Selection.
## 4. Feature Scaling.
## 5. Model Selection.

In [13]:
X_train.sample(1)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
857,1,male,51.0,0,0,26.55,S


In [14]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [15]:
# 1.Dealing Missing values in dataset.
# if we put age insted of index 2, after this imputation dataframe convert into numpy.If we not put index then next pipeline will not able to take input.
tnf1 = ColumnTransformer(transformers=[
    ("impute_age",SimpleImputer(),[2]),
    ("impute_embarked",SimpleImputer(strategy="most_frequent"),[6]),
],remainder="passthrough")

In [16]:
# 2.Dealing with Categorical values
tnf2 = ColumnTransformer(transformers=[
    ("ohe_sex_embarked",OneHotEncoder(sparse=False,
                                      handle_unknown="ignore"),[1,6]),
])

In [17]:
X_train.nunique()

Pclass        3
Sex           2
Age          77
SibSp         7
Parch         7
Fare        207
Embarked      3
dtype: int64

In [18]:
# Feature Scaling
tnf3 = ColumnTransformer(transformers=[
    ("scaling",MinMaxScaler(),slice(0,10))
])

In [19]:
# Feature Selection
tnf4 = SelectKBest(score_func=chi2,k=8)

In [20]:
# Model Selection
tnf5 = LogisticRegression()

# Pipeline

In [21]:
pipe = Pipeline([
    ("tnf1",tnf1),
    ("tnf2",tnf2),
    ("tbf3",tnf3),
    ("tnf4",tnf4),
    ("tnf5",tnf5)
])

# Display Pipeline

In [22]:
from sklearn import set_config
set_config(display="diagram")

# Fit Data Into Pipeline

In [23]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('tnf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('tnf2',
                 ColumnTransformer(transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('tbf3',
                 ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('tnf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7f0c98f4eaf0>)),
                ('tnf5', LogisticRegression())])

In [24]:
pipe.named_steps

{'tnf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'tnf2': ColumnTransformer(transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'tbf3': ColumnTransformer(transformers=[('scaling', MinMaxScaler(),
                                  slice(0, 10, None))]),
 'tnf4': SelectKBest(k=8, score_func=<function chi2 at 0x7f0c98f4eaf0>),
 'tnf5': LogisticRegression()}

In [25]:
# SimpleImputer Calculated this mean values
pipe.named_steps["tnf1"].transformers_[0][1].statistics_

array([29.25635271])

In [26]:
y_pred = pipe.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
accuracy_score(y_test,y_pred)

0.6007462686567164

# Save The Model 

In [29]:
import pickle

In [30]:
pickle.dump(pipe , open("titanic.pkl" , "wb"))

# Cross Validation using Pipeline

In [31]:
from sklearn.model_selection import cross_val_score

In [32]:
cross_val_score(pipe,X_train,y_train,scoring="accuracy",cv=5).mean()

0.6516516129032258

# GridSearch using Pipeline

In [33]:
# gridsearchcv
params = {
    'tnf5__max_depth':[1,2,3,4,5,None]
}

In [34]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

ValueError: Invalid parameter max_depth for estimator LogisticRegression(). Check the list of available parameters with `estimator.get_params().keys()`.

In [ ]:
grid.best_score_

In [ ]:
grid.best_params_

# Load Model

In [ ]:
import pickle
import numpy as np

In [ ]:
pipe = pickle.load(open("titanic.pkl","rb"))

In [ ]:
# For convert input into 2d we do reshape
input = np.array([2, 'male', 31.0, 0, 0, 10.5, 'S'],dtype=object).reshape(1,7)

In [ ]:
pipe.predict(input)